In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datastory import DataStory

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
try:
    con = pesys_utils.open_pen_connection()
except cx_Oracle.DatabaseError:
    con.close()
    con = pesys_utils.open_pen_connection()
    
df_list = pandas_utils.pandas_from_sql('../sql/automatiseringsgrad_test.sql', con, tuning = 50000)
#df_list = pandas_utils.pandas_from_sql('../sql/v8_kun_norge.sql', con)
con.close()
df_list["ÅR-TERTIAL"] = df_list["ÅR-TERTIAL"].apply(lambda x: x[:4] + "-" + x[4:])

In [ ]:
df_list.head()

In [ ]:
PERIODE = "ÅR-MÅNED"

In [ ]:
df_not_batch = df_list[df_list.OPPRETTET_AV != 'Batch']
df_not_batch_2022 = df_not_batch[df_not_batch.ÅR == 2022]

In [ ]:
def group_df_on_var(df, period, var):
    if type(var) is list or type(var) is tuple:
        df = df.groupby([period] + var)[period].count().reset_index(name="ANTALL")
    else:
        df = df.groupby([period, var])[var].count().reset_index(name="ANTALL")

    df = df.merge(df.groupby([period], as_index=False).ANTALL.sum(),
                    left_on=[period],
                    right_on=[period],
                    suffixes=(""," TOTALT")
            )
    df["ANDEL"] = df.ANTALL.divide(df["ANTALL TOTALT"])
    return df

In [ ]:
group_df_on_var(df_list[df_list["ÅR"] == 2022], "ÅR", "OPPRETTET_AV")

In [ ]:
df_count = group_df_on_var(df_not_batch, PERIODE, "AUTOMATISERING")
df_count.head()

In [ ]:
fig1 = px.line(df_count[df_count.AUTOMATISERING=="AUTO"], PERIODE, "ANDEL")
fig1.update_layout(xaxis={'type': 'category'})

In [ ]:
df_opprettet_av = group_df_on_var(df_not_batch, PERIODE, "OPPRETTET_AV")
df_opprettet_av.head()

In [ ]:
fig2 = px.line(df_opprettet_av, PERIODE, "ANDEL", color="OPPRETTET_AV")
fig2.update_layout(xaxis={'type': 'category'})

In [ ]:
df_kravtype = group_df_on_var(df_not_batch, PERIODE, "KRAVTYPE")

In [ ]:
df_kravtype.head()

In [ ]:
fig4 = px.line(df_kravtype, PERIODE, "ANDEL", color="KRAVTYPE")
fig4.update_layout(xaxis={'type': 'category'})

In [ ]:
df_pselv = group_df_on_var(df_not_batch, PERIODE, "SYSTEM")
df_pselv.head()

In [ ]:
fig5 = px.line(df_pselv, PERIODE, "ANDEL", color="SYSTEM")
fig5.update_layout(xaxis={'type': 'category'})

In [ ]:
df_norge = group_df_on_var(df_not_batch[df_not_batch.KRAVTYPE == 'Førstegangsbehandling'], PERIODE, "AUTOMATISERING")
df_norge.head()

In [ ]:
df_auto = df_count[df_count.AUTOMATISERING=="AUTO"]
df_auto_norge = df_norge[df_norge.AUTOMATISERING == 'AUTO']

df_afp = df_not_batch[df_not_batch["SOKT_AFP_PRIVAT"] == "1"]
df_afp_auto = group_df_on_var(df_afp, PERIODE, "AUTOMATISERING")
df_afp_auto = df_afp_auto[df_afp_auto.AUTOMATISERING == "AUTO"]

fig6 = go.Figure()
fig6.add_traces(go.Scatter(x=df_auto_norge[PERIODE], y=df_auto_norge["ANDEL"], name="Førstegangsbehandling"))
fig6.add_traces(go.Scatter(x=df_auto[PERIODE], y=df_auto["ANDEL"], name="Førstegangsbehandling totalt"))
#fig6.add_traces(go.Scatter(x=df_afp_auto[PERIODE], y=df_afp_auto["ANDEL"], name="Førstegangsbehandling med AFP Privat"))
fig6.update_layout(xaxis={'type': 'category'})

In [ ]:
df_auto.tail()

In [ ]:
df_list.head()

In [ ]:
df_pselv_system = group_df_on_var(df_not_batch, PERIODE, ["SYSTEM", "OPPRETTET_AV"])
df_pselv_system["CASE"] = df_pselv_system.SYSTEM + df_pselv_system.OPPRETTET_AV
df_pselv_system.tail(20)

In [ ]:
fig7 = px.line(df_pselv_system, PERIODE, "ANDEL", color="CASE")
fig7.update_yaxes(tickformat='.1%')
fig7.update_layout(xaxis={'type': 'category'})

In [ ]:
fig8 = px.line(df_afp_auto[df_afp_auto.AUTOMATISERING == "AUTO"], PERIODE, "ANDEL")
fig8.update_layout(xaxis={'type': 'category'})

In [ ]:
df_utl = group_df_on_var(df_not_batch, PERIODE, "BODD_ARB_UTL")
fig9 = px.line(df_utl, PERIODE, "ANDEL", color="BODD_ARB_UTL")
fig9.update_layout(xaxis={'type': 'category'})

In [ ]:
df_utl_forstegang = group_df_on_var(df_not_batch[df_not_batch.KRAVTYPE == "Førstegangsbehandling"], PERIODE, "BODD_ARB_UTL")
fig10 = px.line(df_utl_forstegang, PERIODE, "ANDEL", color="BODD_ARB_UTL")
fig10.update_layout(xaxis={'type': 'category'}) 

In [ ]:
df_utl_forstegang = group_df_on_var(df_not_batch[df_not_batch.KRAVTYPE == "Førstegangsbehandling"], PERIODE, "BODD_ARB_UTL")
fig11 = px.line(df_utl_forstegang, PERIODE, "ANTALL", color="BODD_ARB_UTL")
fig11.update_layout(xaxis={'type': 'category'}) 

In [ ]:
df_utl_auto = group_df_on_var(df_not_batch[df_not_batch["BODD_ARB_UTL"] == "1"], PERIODE, "AUTOMATISERING")
fig12 = px.line(df_utl_auto, PERIODE, "ANDEL", color="AUTOMATISERING")
fig12.update_layout(xaxis={'type': 'category'})

# Datastory

In [ ]:
ds = DataStory("Utforskning av automatiseringsgrad i pesys")
ds.markdown("Gjennomgående for alle tall og figurer er at vi ser på fire kravtyper inn alderspensjon: Førstegangsbehandling, Førstegangsbehandling Norge/utland, Førstegangsbehandling bosatt utland og Førstegangsbehandling kun utland. Vi ser bort fra alle krav opprettet av batch.")
ds.markdown("## Utgangspunkt automatiseringsgrad")
ds.plotly(fig1.to_json())

ds.markdown("## Utgangspunkt selvbetjeningsgrad. Trykk bort andre kategorier for å se på andelen opprettet av bruker.")
ds.plotly(fig2.to_json())

ds.markdown("## Andelen av hver kravtype. Legg merke til at andelen 'Førstegangsbehandling' var veldig lav i T3 2022.")
ds.plotly(fig4.to_json())

ds.markdown('## ')

#ds.publish(url="https://nada.intern.nav.no/api")

# Andre metoder

In [ ]:
df_corr = df_not_batch.drop(["ÅR-MÅNED", "ÅR-TERTIAL"], axis=1).apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)

In [ ]:
import seaborn as sns
sns.heatmap(df_corr, 
        xticklabels=df_corr.columns,
        yticklabels=df_corr.columns)